# 🚀 배포된 Foundry Agent 활용하기

Azure AI Foundry에서 배포 및 게시된 Agent를 다양한 방식으로 호출하고 활용하는 방법을 안내합니다.

## ✨ 새로운 Agent를 생성하고, 버전 추가하기

### 📋 단계별 가이드 (Foundry 포탈에서 직접 수행합니다)

#### 1️⃣ Agent 생성
Foundry Portal(Preview)에서 새로운 Agent를 생성합니다.
- 🏷️ **Agent 이름**: `CoffeeExpert`
- 🤖 **배포 Model**: `GPT-4.1`
- 💾 **[저장]** 버튼을 클릭하여 현재 에이전트를 저장합니다
- ✅ [저장] 버튼 좌측에 **v1 저장됨** 이라고 나오는 것을 확인합니다

#### 2️⃣ Agent 지침(Instruction) 설정
Agent의 [지침(Instruction)]란에 다음과 같이 입력합니다:

```text
나는 커피에 대한 전문적인 지식을 제공하는 에이전트입니다. 오직 커피에 대한 정보만을 제공하고, 그 외의 정보는 "죄송합니다. 저는 커피 전문가이다보니 그 밖의 내용은 잘 모르겠습니다."로 대답해야 합니다. 
```

- 💾 **[저장]** 버튼을 클릭하여 현재 에이전트를 저장합니다
- ✅ [저장] 버튼 좌측에 **v2 저장됨** 이라고 나오는 것을 확인합니다

#### 3️⃣ Agent 게시
이제, Agent의 우측 상단에 있는 **[게시]** 버튼을 클릭하여 현재 Agent를 게시합니다.
- ✅ 게시가 완료되면, **[게시됨]** 이라고 나오는 것을 확인합니다
- 📎 세부 정보 화면에서 **[응답 API 엔드포인트]** 를 복사해 둡니다
- 💡 이후 코드에서 이 URL을 사용합니다

## 에이전트 테스트

### 일반적인 HTTP 요청으로 호출하기
클라이언트(예: Postman, requests 라이브러리)를 사용할 때는 이 토큰을 `Authorization` 헤더에 `Bearer {accessToken}` 형식으로 포함시켜 API를 호출해야 합니다.

#### 🔐 Access Token 발급하기

> **💡 참고사항**  
> HTTP 응답 테스트를 위해서는 임시 Access Token을 발급받아야 합니다.  
> 실제 코드에서는 `DefaultAzureCredential` 나 `AzureCliCredential`을 사용하여 Managed Identity 방식으로 인증을 처리할 수 있기에 토큰이 필요없지만, Postman이나 requests 라이브러리를 사용할 때는 토큰이 필요합니다.

하기 코드를 수행하고 `accessToken` 값을 얻습니다.

In [ ]:
!az account get-access-token --resource https://ai.azure.com

In [ ]:
import requests

publised_url = "<응답 API 엔드포인트>"
accessToken = "<위에서 발급 받은 Access Token>"
#publised_url = "https://aif-taeyok.services.ai.azure.com/api/projects/proj-default/applications/CoffeeExpert/protocols/openai/responses?api-version=2025-11-15-preview"

headers = {
    "Content-Type":"application/json", 
    "Authorization":f"Bearer {accessToken}"
}

input = {"input":"지금 삼성전자 주식을 사도 될까?"}
response = requests.post(publised_url, headers=headers, json=input)

if(response.status_code != 200):
    print("Error:", response.status_code, response.text)
    exit(1)
else:
    text = response.json()['output'][0]['content'][0]['text']
    print("결과:", text)

결과: 죄송합니다. 저는 커피전문가이다보니 그와 관련된 내용은 잘 모르겠습니다.


### 🔧 OpenAI SDK 사용해서 Azure AI Foundry Agent 호출하기

OpenAI SDK를 활용하면 간편하게 배포된 Agent를 호출할 수 있습니다.  
이 경우에는 Access Token을 발급받을 필요 없이, SDK가 자동으로 인증을 처리합니다.

In [5]:
from openai import OpenAI 
from azure.identity import DefaultAzureCredential, get_bearer_token_provider 

# edit base_url with your <foundry-resource-name>, <project-name>, and <app-name>
openai = OpenAI(
    api_key=get_bearer_token_provider(DefaultAzureCredential(), "https://ai.azure.com/.default"),
    base_url="https://aif-taeyok.services.ai.azure.com/api/projects/proj-default/applications/CoffeeExpert/protocols/openai",
    default_query = {"api-version": "2025-11-15-preview"}
)

response = openai.responses.create( 
  input="아메리카노와 롱블랙의 차이는?", 
) 
print(f"Response output: {response.output_text}")
print("-"*70)
response = openai.responses.create( 
  input="지금 삼성전자 주식을 사도 될까?", 
) 
print(f"Response output: {response.output_text}")

Response output: 아메리카노와 롱블랙은 모두 에스프레소와 물을 혼합한 커피이지만 방식에 차이가 있습니다.

- **아메리카노**: 에스프레소에 뜨거운 물을 추가하여 만듭니다. 일반적으로 에스프레소를 먼저 추출한 후 물을 붓기 때문에 크레마가 흐려질 수 있습니다.

- **롱블랙**: 뜨거운 물에 에스프레소를 추가하여 만듭니다. 이 방식은 크레마가 더 잘 유지되며 풍미가 더욱 진하게 느껴질 수 있습니다.

이 두 종류의 커피는 에스프레소와 물의 비율이 비슷하지만, 만드는 방법에 따라 풍미와 식감이 다르게 느껴질 수 있습니다.
----------------------------------------------------------------------
Response output: 죄송합니다. 저는 커피전문가이다보니 그와 관련된 내용은 잘 모르겠습니다.


### 🌐 Foundry Agent SDK를 사용해서 호출하기

> **📌 중요**  
>
> 다음의 코드는 Foundry Portal(preview)에 배포된 에이전트를 사용해서 사용자의 요청에 응답하는 방법을 보여줍니다.  
> **(신규 포탈에 있는 Agent 호출 방법입니다. Classic 포탈의 Agent 호출 방법은 [AzureAIFoundryAgent.ipynb](AzureAIFoundryAgent.ipynb) 참조)**

In [6]:
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AgentReference

# 설정 정보
PROJECT_ENDPOINT =  os.environ["AZURE_AI_PROJECT_ENDPOINT"]
AGENT_NAME = "CoffeeExpert"

# 클라이언트를 초기화하고, 에이전트를 가져옵니다.
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=DefaultAzureCredential())
agent = client.agents.get(agent_name=AGENT_NAME)

# OpenAI 클라이언트를 가져오고, Agent 정보를 더해서 메시지를 보냅니다.
openai_client = client.get_openai_client()
response = openai_client.responses.create(
    input=[{"role": "user", "content": "겨울 아침에 어울리는 커피는?"}],
    extra_body={"agent": AgentReference(name=agent.name).as_dict()}
)

print(f"Agent response: {response.output_text}")


Agent response: 겨울 아침에는 따뜻하고 풍부한 맛을 가진 커피가 좋습니다. 몇 가지 추천해드리자면:

1. **카푸치노**: 벨벳 같은 스팀 밀크와 에스프레소가 조화를 이루어 부드럽고 진한 맛을 제공합니다.

2. **카페 모카**: 초콜릿 시럽과 에스프레소가 결합되어 달콤하고 따뜻한 느낌을 줍니다.

3. **라떼**: 풍부한 스팀 밀크와 부드러운 에스프레소가 추운 아침을 따뜻하게 해줍니다.

4. **프렌치 프레스 커피**: 조금 더 강하고 깊은 맛을 원하는 경우, 프렌치 프레스 커피를 통해 추출한 풍부한 향미를 즐길 수 있습니다.

각 커피에 시나몬이나 너트메그를 뿌려 겨울 특유의 분위기를 더할 수도 있습니다.


### 📊 Observability - 대화 이력 및 사용량 모니터링

**✨ 핵심** : Foundry Agent SDK를 사용해서 Agent를 호출하게 되면, 모든 대화 이력과 사용량이 Foundry 포탈에 기록되며, 완전한 Observability를 제공합니다.

Foundry Portal에서 Agent의 **[추적]** 탭을 통해서 다음과 같이 대화 이력과 사용량을 모니터링할 수 있습니다.

📈 **추적 대시보드**  
![MAF-inoke-log1.png](./images/MAF-inoke-log1.png)  

📊 **상세 분석**  
![MAF-inoke-log1.png](./images/MAF-inoke-log2.png)

### 🎯 특정 배포버전의 Agent 호출하기

> **🔖 버전 관리**  
>
> 다음의 코드는 배포된 에이전트의 특정 배포버전을 지정하여 호출하는 방법을 보여줍니다.  
버전을 1과 2로 바꿔가면 실행을 해 봅니다. 1의 경우는 지침이 입력되기 전이기에 모든 질문에 응답을 할 것입니다.  
버전 2로 바꾸면, 지침이 지정된 다음이기에 커피 관련 질문에만 응답을 하게 됩니다.

In [7]:
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AgentReference

# Configuration
PROJECT_ENDPOINT =  os.environ["AZURE_AI_PROJECT_ENDPOINT"]
AGENT_NAME = "CoffeeExpert"
AGENT_VERSION = "1"  # Optional: specify version, or use latest

client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=DefaultAzureCredential())
agent = client.agents.get(agent_name=AGENT_NAME)

openai_client = client.get_openai_client()
response = openai_client.responses.create(
    input=[{"role": "user", "content": "지금 현대자동차 주식을 사도 될까?"}],
    extra_body={"agent": AgentReference(name=agent.name, version=AGENT_VERSION).as_dict()}
)

print(f"Agent response: {response.output_text}")


Agent response: 투자 결정을 내릴 때 몇 가지 중요한 요소를 고려해야 합니다:

1. **분석**: 현대자동차의 최근 실적 보고서, 매출, 순이익, 시장 점유율 등을 분석하세요.

2. **시장 트렌드**: 자동차 산업 전반의 트렌드와 전기차, 자율주행차 등 현대자동차가 집중하는 분야의 성장 가능성을 확인하세요.

3. **경제 상황**: 국내외 경제 상황과 금리, 환율 변동 등의 영향을 점검하세요.

4. **리스크**: 특정 주식에 대한 투자 리스크를 평가하고, 분산 투자를 통해 리스크를 관리하세요.

5. **전문가 의견**: 주식 애널리스트의 의견과 목표 주가를 참고하세요.

결국 본인의 투자 목표와 리스크 감내 수준에 따라 결정해야 합니다. 전문가 상담을 권장합니다.
